In [1]:
# Imports you may need
from dataloader import *
from finance import *
import pandas as pd

from finance import stock, compare
from quotebankexploration import *
from wikipedia import *

import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py


from ipywidgets import interactive, HBox, VBox, Checkbox

%load_ext autoreload
%autoreload 2

tqdm.pandas()

In [2]:
# Get some data frames
wiki_data = concat_wiki_files()
filtered_quotes = get_filtered_quotes()
speakers_id = pd.DataFrame(get_speakers_ids(filtered_quotes))
speakers_labels = get_speakers_labels_one_file()

In [3]:
quotes_test = filtered_quotes.sample(100).reset_index(drop = True)
id_test = pd.DataFrame(get_speakers_ids(quotes_test))
labels_test = find_labels(id_test, wiki_data)

  0%|          | 0/39 [00:00<?, ?it/s]

In [4]:
for year in range(2015, 2021):
    labels_test[str(year)] = labels_test.label.apply(lambda label: get_page_views_per_year(label, year))
    print(year)

2015
2016
2017
2018
2019
2020


In [5]:
quotes_test['year'] = quotes_test.date.apply(lambda date: date.year)

In [6]:
yolo = quotes_test.set_index('speaker').join(labels_test.set_index('speaker'), lsuffix="_left", rsuffix="_right").reset_index()
display(yolo)

,speaker,quoteID,quotation,qids_left,date,numOccurrences,probas,urls,phase,year,qids_right,label,2015,2016,2017,2018,2019,2020
0,andreas sennheiser,2016-02-15-093756,"They own a big audio company now, and they hav...",[Q23760757],2016-02-15 13:45:36,1,"[[Andreas Sennheiser, 0.7337], [None, 0.2663]]",[http://twice.com/why-switch-lightning-coming/...,E,2016,[Q23760757],Andreas Sennheiser,905.0,2411.0,2181.0,243.0,201.0,146.0
1,angela ahrendts,2013-10-15-051416,"oversight of the strategic direction, expansio...",[Q529233],2013-10-15 06:45:35,1,"[[Angela Ahrendts, 0.7396], [None, 0.2256], [T...",[http://forums.appleinsider.com/t/160168/apple...,D,2013,[Q529233],Angela Ahrendts,73207.0,131075.0,153051.0,186307.0,142815.0,75608.0
2,angela ahrendts,2013-10-15-016538,"have oversight of the strategic direction, exp...",[Q529233],2013-10-15 11:47:00,1,"[[Angela Ahrendts, 0.7416], [None, 0.2584]]",[http://www.v3.co.uk/v3-uk/news/2300691/apple-...,D,2013,[Q529233],Angela Ahrendts,73207.0,131075.0,153051.0,186307.0,142815.0,75608.0
3,ankit jain,2014-10-09-018633,Google doesn't want to make the same mistake t...,[Q58266450],2014-10-09 20:39:05,4,"[[Ankit Jain, 0.852], [None, 0.1325], [Sundar ...",[http://blogs.wsj.com/digits/2014/10/09/google...,E,2014,[Q58266450],None,NaN,NaN,NaN,NaN,NaN,NaN
4,anthony daniels,2017-08-06-032165,"In 1901, they only had one copy of the bill, b...","[Q30122960, Q441685]",2017-08-06 05:00:00,1,"[[Anthony Daniels, 0.7084], [None, 0.1891], [D...",[http://www.timesdaily.com/news/state-capital/...,E,2017,"[Q30122960, Q441685]",Anthony Daniels,651684.0,600738.0,429151.0,283991.0,526704.0,442344.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,tim cook,2012-12-06-029131,"i loved steve dearly, and miss him dearly. and...","[Q1404825, Q265852, Q7803347, Q7803348]",2012-12-06 20:11:00,1,"[[tim cook, 0.5015], [steve jobs, 0.2011], [br...",[http://sys-con.com/node/2475040],C,2012,"[Q1404825, Q265852, Q7803347, Q7803348]",Tim Cook,663329.0,1414024.0,1201433.0,1272243.0,1329793.0,1483327.0
96,timothy d. cook,2014-04-24-086762,We gained smartphone share in many developed a...,[Q265852],2014-04-24 11:54:00,2,"[[Timothy D. Cook, 0.9208], [None, 0.0792]]",[http://us.rd.yahoo.com/finance/external/tsmfe...,D,2014,[Q265852],Tim Cook,663329.0,1414024.0,1201433.0,1272243.0,1329793.0,1483327.0
97,tracy morgan,2010-09-07-092341,we are so excited to introduce the true `runne...,"[Q14542, Q27119857]",2010-09-07 19:27:48,4,"[[tracy morgan, 0.323], [lance armstrong, 0.26...",[http://earthtimes.org/articles/press/new-nike...,C,2010,"[Q14542, Q27119857]",Tracy Morgan,839072.0,845475.0,1008968.0,911127.0,1066569.0,872967.0
98,tyler perry,2019-10-16-034761,I get a FaceTime call and she's holding up the...,[Q686301],2019-10-16 16:22:54,1,"[[Tyler Perry, 0.8326], [None, 0.1674]]",[http://www.cheatsheet.com/entertainment/is-ty...,E,2019,[Q686301],Tyler Perry,725588.0,1659077.0,1491506.0,1608420.0,1883998.0,2509799.0


In [7]:
def scoring(quotes):
    try:
        score = quotes[str(quotes['year'])]
    except:
        score = None
    return score

shit = yolo.copy()
shit['score'] = yolo.apply(scoring, axis = 1)
shit = shit.drop(['2015', '2016', '2017', '2018', '2019', '2020', 'year'], axis = 1)
display(shit)

,speaker,quoteID,quotation,qids_left,date,numOccurrences,probas,urls,phase,qids_right,label,score
0,andreas sennheiser,2016-02-15-093756,"They own a big audio company now, and they hav...",[Q23760757],2016-02-15 13:45:36,1,"[[Andreas Sennheiser, 0.7337], [None, 0.2663]]",[http://twice.com/why-switch-lightning-coming/...,E,[Q23760757],Andreas Sennheiser,2411.0
1,angela ahrendts,2013-10-15-051416,"oversight of the strategic direction, expansio...",[Q529233],2013-10-15 06:45:35,1,"[[Angela Ahrendts, 0.7396], [None, 0.2256], [T...",[http://forums.appleinsider.com/t/160168/apple...,D,[Q529233],Angela Ahrendts,NaN
2,angela ahrendts,2013-10-15-016538,"have oversight of the strategic direction, exp...",[Q529233],2013-10-15 11:47:00,1,"[[Angela Ahrendts, 0.7416], [None, 0.2584]]",[http://www.v3.co.uk/v3-uk/news/2300691/apple-...,D,[Q529233],Angela Ahrendts,NaN
3,ankit jain,2014-10-09-018633,Google doesn't want to make the same mistake t...,[Q58266450],2014-10-09 20:39:05,4,"[[Ankit Jain, 0.852], [None, 0.1325], [Sundar ...",[http://blogs.wsj.com/digits/2014/10/09/google...,E,[Q58266450],None,NaN
4,anthony daniels,2017-08-06-032165,"In 1901, they only had one copy of the bill, b...","[Q30122960, Q441685]",2017-08-06 05:00:00,1,"[[Anthony Daniels, 0.7084], [None, 0.1891], [D...",[http://www.timesdaily.com/news/state-capital/...,E,"[Q30122960, Q441685]",Anthony Daniels,429151.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,tim cook,2012-12-06-029131,"i loved steve dearly, and miss him dearly. and...","[Q1404825, Q265852, Q7803347, Q7803348]",2012-12-06 20:11:00,1,"[[tim cook, 0.5015], [steve jobs, 0.2011], [br...",[http://sys-con.com/node/2475040],C,"[Q1404825, Q265852, Q7803347, Q7803348]",Tim Cook,NaN
96,timothy d. cook,2014-04-24-086762,We gained smartphone share in many developed a...,[Q265852],2014-04-24 11:54:00,2,"[[Timothy D. Cook, 0.9208], [None, 0.0792]]",[http://us.rd.yahoo.com/finance/external/tsmfe...,D,[Q265852],Tim Cook,NaN
97,tracy morgan,2010-09-07-092341,we are so excited to introduce the true `runne...,"[Q14542, Q27119857]",2010-09-07 19:27:48,4,"[[tracy morgan, 0.323], [lance armstrong, 0.26...",[http://earthtimes.org/articles/press/new-nike...,C,"[Q14542, Q27119857]",Tracy Morgan,NaN
98,tyler perry,2019-10-16-034761,I get a FaceTime call and she's holding up the...,[Q686301],2019-10-16 16:22:54,1,"[[Tyler Perry, 0.8326], [None, 0.1674]]",[http://www.cheatsheet.com/entertainment/is-ty...,E,[Q686301],Tyler Perry,1883998.0


# Feelings

***

In [8]:
analyzer = SentimentIntensityAnalyzer()
# determine the sentiment of a quote in a corpus (positive, negative or neutral)
def sentiment(quote) : 
    vs = analyzer.polarity_scores(quote)['compound']
    if (vs >=0.05) :
        return 1
    if (vs <= - 0.05) :
        return -1
    else : return 0

fuck = shit.copy()
fuck['sentiment'] = fuck['quotation'].apply(sentiment) 
df_sent = fuck.groupby(['sentiment']).sum().reset_index()
display(fuck)

,speaker,quoteID,quotation,qids_left,date,numOccurrences,probas,urls,phase,qids_right,label,score,sentiment
0,andreas sennheiser,2016-02-15-093756,"They own a big audio company now, and they hav...",[Q23760757],2016-02-15 13:45:36,1,"[[Andreas Sennheiser, 0.7337], [None, 0.2663]]",[http://twice.com/why-switch-lightning-coming/...,E,[Q23760757],Andreas Sennheiser,2411.0,0
1,angela ahrendts,2013-10-15-051416,"oversight of the strategic direction, expansio...",[Q529233],2013-10-15 06:45:35,1,"[[Angela Ahrendts, 0.7396], [None, 0.2256], [T...",[http://forums.appleinsider.com/t/160168/apple...,D,[Q529233],Angela Ahrendts,NaN,0
2,angela ahrendts,2013-10-15-016538,"have oversight of the strategic direction, exp...",[Q529233],2013-10-15 11:47:00,1,"[[Angela Ahrendts, 0.7416], [None, 0.2584]]",[http://www.v3.co.uk/v3-uk/news/2300691/apple-...,D,[Q529233],Angela Ahrendts,NaN,0
3,ankit jain,2014-10-09-018633,Google doesn't want to make the same mistake t...,[Q58266450],2014-10-09 20:39:05,4,"[[Ankit Jain, 0.852], [None, 0.1325], [Sundar ...",[http://blogs.wsj.com/digits/2014/10/09/google...,E,[Q58266450],None,NaN,-1
4,anthony daniels,2017-08-06-032165,"In 1901, they only had one copy of the bill, b...","[Q30122960, Q441685]",2017-08-06 05:00:00,1,"[[Anthony Daniels, 0.7084], [None, 0.1891], [D...",[http://www.timesdaily.com/news/state-capital/...,E,"[Q30122960, Q441685]",Anthony Daniels,429151.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,tim cook,2012-12-06-029131,"i loved steve dearly, and miss him dearly. and...","[Q1404825, Q265852, Q7803347, Q7803348]",2012-12-06 20:11:00,1,"[[tim cook, 0.5015], [steve jobs, 0.2011], [br...",[http://sys-con.com/node/2475040],C,"[Q1404825, Q265852, Q7803347, Q7803348]",Tim Cook,NaN,1
96,timothy d. cook,2014-04-24-086762,We gained smartphone share in many developed a...,[Q265852],2014-04-24 11:54:00,2,"[[Timothy D. Cook, 0.9208], [None, 0.0792]]",[http://us.rd.yahoo.com/finance/external/tsmfe...,D,[Q265852],Tim Cook,NaN,1
97,tracy morgan,2010-09-07-092341,we are so excited to introduce the true `runne...,"[Q14542, Q27119857]",2010-09-07 19:27:48,4,"[[tracy morgan, 0.323], [lance armstrong, 0.26...",[http://earthtimes.org/articles/press/new-nike...,C,"[Q14542, Q27119857]",Tracy Morgan,NaN,1
98,tyler perry,2019-10-16-034761,I get a FaceTime call and she's holding up the...,[Q686301],2019-10-16 16:22:54,1,"[[Tyler Perry, 0.8326], [None, 0.1674]]",[http://www.cheatsheet.com/entertainment/is-ty...,E,[Q686301],Tyler Perry,1883998.0,0


# Get back 2015 / 2016

***

In [9]:
prout = pd.read_pickle('data/wiki_speaker_attributes/speakers_pageviews_2015')
prout_ = pd.read_pickle('data/wiki_speaker_attributes/speakers_pageviews_2016')
prout['2016'] = prout_['2016']

In [10]:
quotes_prout = filtered_quotes.copy()
quotes_prout['year'] = quotes_prout.date.apply(lambda date: date.year)
df = quotes_prout.set_index('speaker').join(prout.set_index('speaker'), lsuffix="_left", rsuffix="_right").reset_index()

In [11]:
df_ = df[df.year >= 2015].copy()
df_ = df_[df_.year < 2017]
df_['absolute_score'] = df.apply(scoring, axis = 1)
df_ = df_.drop(['2015', '2016', 'year'], axis = 1)

In [12]:
df_final = df_.copy()
df_final['sentiment'] = df_final['quotation'].progress_apply(sentiment)

  0%|          | 0/48119 [00:00<?, ?it/s]

In [13]:
def func_pos(row):
    score = 0
    if row.sentiment > 0:
        score = row.score
    return score

def func_neg(row):
    score = 0
    if row.sentiment < 0:
        score = row.score
    return score


df_final['score_pos'] = df_final.progress_apply(func_pos, axis = 1)
df_final['score_neg'] = df_final.progress_apply(func_neg, axis = 1)
df_final_ = df_final[['date', 'score_pos', 'score_neg']].dropna().groupby(['date']).sum()
df_final_ = df_final_.reset_index(drop = False)
display(df_final_)

  0%|          | 0/48119 [00:00<?, ?it/s]

AttributeError: 'Series' object has no attribute 'score'

In [ ]:
trace1 = go.Bar(
    x=ddf_final_.date,
    y=df[num],
    name=num,
    marker=dict(
        color='rgb(34,163,192)'
               )
)
trace2 = go.Scatter(
    x=df[cat],
    y=df['cumulative_perc'],
    name='Cumulative Percentage',
    yaxis='y2'

)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.add_trace(trace2,secondary_y=True)
fig['layout'].update(height = 600, width = 800, title = title,xaxis=dict(
      tickangle=-90
    ))
iplot(fig)



fig = px.bar(df_final_, x='date', y='score_pos')
fig.update_layout(
title={
        'text' : 'Distribution of the positive quotes according to time',
        'x':0.5,
        'xanchor': 'center'},
xaxis_title_text='date', # xaxis label
yaxis_title_text='frequency of positive quotes', # yaxis label
bargap=0.2, # gap between bars of adjacent location coordinates
bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.update_traces(marker_line_width = 0,
                selector=dict(type="bar"))

fig.update_layout(bargap=0.1,
                bargroupgap = 0,
                )
fig.show()

# Graphics

***

In [ ]:
# separate the positive and negative quotes
pos_quotes = fuck[fuck['sentiment'] == 'Positive']
neg_quotes = fuck[fuck['sentiment'] == 'Negative']
neut_quotes = fuck[fuck['sentiment'] == 'Neutral']

In [ ]:
# plot the distribution of the neutral quotes according to time
neut_per_day = pd.DataFrame(neut_quotes.groupby(neut_quotes.date.dt.date).count()['sentiment'])
neut_per_day.index.rename('Date')
neut_per_day.reset_index(inplace=True)

# plot the distribution of the positive quotes according to time
pos_per_day = pd.DataFrame(pos_quotes.groupby(pos_quotes.date.dt.date).count()['sentiment'])
pos_per_day.index.rename('Date')
pos_per_day.reset_index(inplace=True)

# plot the distribution of the negative quotes according to time
neg_per_day = pd.DataFrame(neg_quotes.groupby(neg_quotes.date.dt.date).count()['sentiment'])
neg_per_day.index.rename('Date')
neg_per_day.reset_index(inplace=True)

In [ ]:
fig = px.bar(pos_per_day, x=pos_per_day['date'], y=pos_per_day['sentiment'])
fig.update_layout(
title={
        'text' : 'Distribution of the positive quotes according to time',
        'x':0.5,
        'xanchor': 'center'},
xaxis_title_text='date', # xaxis label
yaxis_title_text='frequency of positive quotes', # yaxis label
bargap=0.2, # gap between bars of adjacent location coordinates
bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.update_traces(marker_line_width = 0,
                selector=dict(type="bar"))

fig.update_layout(bargap=0.1,
                bargroupgap = 0,
                )
fig.show()

In [ ]:
fig = px.bar(neg_per_day, x=neg_per_day['date'], y=neg_per_day['sentiment'])
fig.update_layout(
title={
        'text' : 'Distribution of the negative quotes according to time',
        'x':0.5,
        'xanchor': 'center'},
xaxis_title_text='date', # xaxis label
yaxis_title_text='frequency of negative quotes', # yaxis label
bargap=0.2, # gap between bars of adjacent location coordinates
bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.update_traces(marker_line_width = 0,
                selector=dict(type="bar"))

fig.update_layout(bargap=0.1,
                bargroupgap = 0,
                )
fig.show()

In [ ]:
# Initialize figure
all_quotes_per_day = pos_per_day.merge(neg_per_day, how="right", on="date").merge(neut_per_day, how="left", on="date")
all_quotes_per_day.date = all_quotes_per_day.date.apply(lambda x : str(x))
all_quotes_per_day.rename({"sentiment_x": "Positive", "sentiment_y": "Negative", "sentiment":"Neutral"},axis=1,inplace=True)
all_quotes_per_day['All'] = all_quotes_per_day.Positive + all_quotes_per_day.Negative + all_quotes_per_day.Neutral


fig = go.Figure()
ymax = all_quotes_per_day.All.max()

# Add Traces

fig.add_trace(
    go.Bar(x=all_quotes_per_day.date,
            y=all_quotes_per_day.All,
            name="All",
            marker_color="blue"))

fig.add_trace(
    go.Bar(x=all_quotes_per_day.date,
            y=all_quotes_per_day.Negative,
            name="Negative",
                visible=False,
            marker_color="red"))

fig.add_trace(
    go.Bar(x=neut_per_day.date,
            y=all_quotes_per_day.Positive,
            name="Positive",
            visible = False,
            marker_color="green"))

fig.add_trace(
    go.Bar(x=neut_per_day.date,
            y=all_quotes_per_day.Neutral,
            name="Neutral",
                visible=False,
            marker_color="gray"))

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            x=0.57,
            y=1.2,
            buttons=list([
                dict(label="All",
                    method="update",
                    args=[{"visible": [True, False, False, False]},
                        {"title": "All quotes",
                            "annotations": []}]),
                dict(label="Negative",
                    method="update",
                    args=[{"visible": [False, True, False, False]},
                        {"title": "Negative quotes"}]),
                dict(label="Positive",
                    method="update",
                    args=[{"visible": [False, False, True, False]},
                        {"title": "Positive quotes"}]),
                dict(label="Neutral",
                    method="update",
                    args=[{"visible": [False, False, False, True]},
                        {"title": "Neutral quotes"}]),
            ]),
        )
    ])

# Set title
fig.update_layout(
    title_text="All quotes",
    xaxis_domain=[0.05, 1.0],
    yaxis_range =[0,ymax],
    xaxis_title_text='Date', # xaxis label
    yaxis_title_text='Frequency of quotes', # yaxis label
    bargap=0.1,
    bargroupgap = 0
)
fig.update_traces(marker_line_width = 0,
                selector=dict(type="bar"))
fig.show()

In [ ]:
from task3 import *
task3()

***

In [ ]:
display(speakers_labels.head(10))

In [ ]:
print(get_page_views_per_year('Shameel Joosub', 2015))

In [ ]:
yolo = filtered_quotes.copy()
yolo['year'] = filtered_quotes.date.apply(lambda date: date.year)
display(yolo)